In [1]:
import warnings
warnings.filterwarnings('ignore')

import keras
from keras.models import load_model
from keras import backend as K

#from utils import load_test_data



Using TensorFlow backend.


In [2]:
ensembling_models = {
    'keras_whereami_trained_model_c' :{'img_size':139},
    'keras_whereami_trained_model_v' :{'img_size': 128}
}

In [3]:
import ntpath
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

test_path = os.path.join(os.getcwd(), 'test')
test_listc = glob.glob(os.path.join(test_path, '*_c.png'))
test_id = [path_leaf(s).strip('_c.png') for s in test_listc]
test_listv=[test_listc[x].replace('_c','_v') for x in range(len(test_listc))]
print(len(test_listv))



659


In [4]:
print(test_listc[:5])
print(test_listv[:5])

['/home/jovyan/edx/test/2051_c.png', '/home/jovyan/edx/test/2569_c.png', '/home/jovyan/edx/test/2121_c.png', '/home/jovyan/edx/test/2043_c.png', '/home/jovyan/edx/test/2504_c.png']
['/home/jovyan/edx/test/2051_v.png', '/home/jovyan/edx/test/2569_v.png', '/home/jovyan/edx/test/2121_v.png', '/home/jovyan/edx/test/2043_v.png', '/home/jovyan/edx/test/2504_v.png']


In [5]:
def load_testdata(n_size):
    test_data_list =[]
    for i in range(len(test_listc)):
        if(n_size==139):
            img = cv2.imread(test_listc[i])
        else:
            img = cv2.imread(test_listv[i])
 
        img = cv2.resize(img, (img_size, img_size))
        img = img.astype(np.float32)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img - np.array([123.68, 116.779, 103.939])
        #img = img[:,:,::-1]  # RGB to BGR
        img = (img - np.min(img)) / np.max(img)
        test_data_list.append(np.asarray(img))

    test_data = np.array(test_data_list)    
    print(test_data.shape)
    return test_data

In [6]:
for key in ensembling_models.keys():
    print(key)
    img_size = ensembling_models[key]['img_size']
    model = load_model('./saved_models/{}.h5'.format(key))
    X_test= load_testdata(img_size)
    ensembling_models[key]['y_test_pred_prob'] = model.predict(X_test)

0.79_c_keras_whereami_trained_model
(659, 139, 139, 3)
0.796_v_keras_whereami_trained_model
(659, 128, 128, 3)


In [7]:
num_models = float(len(ensembling_models.keys()))
y_test_pred_prob = 0
for key in ensembling_models.keys():
    y_test_pred_prob += ensembling_models[key]['y_test_pred_prob']

y_test_pred_prob /= num_models

In [9]:
y_test_pred = y_test_pred_prob.argmax(axis=-1)

In [11]:
import numpy as np
import pandas as pd
df = pd.DataFrame( {"id": test_id, "appliance": pd.Series(y_test_pred)} )
df = df[['id', 'appliance']]
df=df.sort_values (["id"])
#df=df.sort_values (["id"],ascending=True)
#df.to_csv('submission_data0.csv', index = False)
df.to_csv('ensemble.csv', index = False)